In [1]:
%matplotlib inline

from __future__ import print_function
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt

rcParams['figure.figsize'] = (13, 6)

import nilmtk
from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation
from nilmtk.utils import print_dict
from nilmtk.metrics import f1_score

import pandas as pd
import numpy as np
from nilmtk.measurement import measurement_columns, AC_TYPES

import pytz


# oth-regensburg-1

In [2]:
df = pd.read_csv("oth-regensburg-1/oth-regensburg-1.csv")
print(df.shape)
df.head()

(1550472, 27)


,Id,Name,Serial,DeviceEnergyType,MeterSubType,ActivePower,CounterReading,CounterReadingT1,CounterReadingT2,CounterReadingExport,...,VoltageL3,Current,CurrentL1,CurrentL2,CurrentL3,PowerFactor,PowerFactorL1,PowerFactorL2,PowerFactorL3,DigitalInput1
0,43043308-a366-4af4-881c-6bae4a0a0dc3,smart-me 6011785 - Allgemein,6011785,MeterTypeElectricity,NaN,0.133324,3016.232117,3016.232117,0.0,7.343095,...,234.6,NaN,0.361,0.614,0.172,0.333,0.333,0.690,0.152,NaN
1,aa680cb9-5092-4483-84fe-a5df826a55c4,smart-me 6011209 Batterie,6011209,MeterTypeElectricity,NaN,-0.008716,-1761.777895,-1761.777895,0.0,12496.138372,...,0.0,NaN,-0.057,0.000,0.000,0.650,0.650,0.000,0.000,NaN
2,2c6913bf-c807-4592-a148-a891b6174aae,Gesamtverbrauch,637686123692284276,13,MeterSubTypeElectricity,0.750239,75000.402125,75000.402125,0.0,7.343097,...,NaN,0.0,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN
3,e8755559-b3da-4a71-baa6-109fd1304248,smart-me 6011227 - Heizung,6011227,MeterTypeElectricity,NaN,0.047855,36439.485954,36439.485954,0.0,0.000000,...,234.6,NaN,0.321,0.000,0.000,0.642,0.642,0.000,0.000,NaN
4,a9dad66e-8a0b-457f-8cd8-644aeddee557,smart-me 6011505 - Wohnung 1,6011505,MeterTypeElectricity,NaN,0.052729,4050.142234,4050.142234,0.0,0.000000,...,234.6,NaN,0.701,0.467,0.091,0.054,0.054,0.402,0.000,NaN


In [3]:
Wohnung1 = df[df.Name.str.contains("Wohnung 1")][['ActivePower','ValueDate']]

In [4]:
Wohnung1.head()

,ActivePower,ValueDate
4,0.052729,08/01/2023 00:00:03
16,0.053102,08/01/2023 00:00:05
28,0.052937,08/01/2023 00:00:07
40,0.052926,08/01/2023 00:00:09
52,0.053356,08/01/2023 00:00:10


In [5]:
Wohnung1['ValueDate'] = pd.to_datetime(Wohnung1['ValueDate'], format='%m/%d/%Y %H:%M:%S')
Wohnung1['ValueDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
tz = pytz.timezone('Europe/Berlin')
Wohnung1['ValueDate'] = Wohnung1['ValueDate'].dt.tz_localize(tz)
Wohnung1.head()

,ActivePower,ValueDate
4,0.052729,2023-08-01 00:00:03+02:00
16,0.053102,2023-08-01 00:00:05+02:00
28,0.052937,2023-08-01 00:00:07+02:00
40,0.052926,2023-08-01 00:00:09+02:00
52,0.053356,2023-08-01 00:00:10+02:00


In [6]:
Wohnung1.ActivePower = Wohnung1.ActivePower * 1000

In [7]:
Wohnung1.set_index('ValueDate', inplace=True)
Wohnung1.index.name = None
# Wohnung1.reset_index(inplace=True)
Wohnung1.head()

,ActivePower
2023-08-01 00:00:03+02:00,52.729
2023-08-01 00:00:05+02:00,53.102
2023-08-01 00:00:07+02:00,52.937
2023-08-01 00:00:09+02:00,52.926
2023-08-01 00:00:10+02:00,53.356


In [8]:
Wohnung1.values

array([[52.729],
       [53.102],
       [52.937],
       ...,
       [95.077],
       [95.45 ],
       [95.6  ]])

In [9]:
Wohnung1.index

DatetimeIndex(['2023-08-01 00:00:03+02:00', '2023-08-01 00:00:05+02:00',
               '2023-08-01 00:00:07+02:00', '2023-08-01 00:00:09+02:00',
               '2023-08-01 00:00:10+02:00', '2023-08-01 00:00:13+02:00',
               '2023-08-01 00:00:14+02:00', '2023-08-01 00:00:16+02:00',
               '2023-08-01 00:00:18+02:00', '2023-08-01 00:00:20+02:00',
               ...
               '2023-08-31 10:17:28+02:00', '2023-08-31 10:18:06+02:00',
               '2023-08-31 10:18:44+02:00', '2023-08-31 10:19:23+02:00',
               '2023-08-31 10:20:01+02:00', '2023-08-31 10:20:39+02:00',
               '2023-08-31 10:21:18+02:00', '2023-08-31 10:21:56+02:00',
               '2023-08-31 10:22:35+02:00', '2023-08-31 10:23:13+02:00'],
              dtype='datetime64[ns, Europe/Berlin]', length=129206, freq=None)

In [10]:
smartme_df = pd.DataFrame(data=Wohnung1.values, index=Wohnung1.index, dtype=np.float32,
                       columns=measurement_columns([('power', 'active')]))

In [11]:
smartme_df.head()

physical_quantity,power
type,active
2023-08-01 00:00:03+02:00,52.729000
2023-08-01 00:00:05+02:00,53.102001
2023-08-01 00:00:07+02:00,52.937000
2023-08-01 00:00:09+02:00,52.925999
2023-08-01 00:00:10+02:00,53.355999


In [13]:
smartme_df.index.name

# create smartme.h5

In [12]:
from generate_smartme import create_smartme_hdf5

In [13]:
create_smartme_hdf5(smartme_df, 'smartme.h5')

Saving building1/elec/meter1
Done converting YAML metadata to HDF5!
